In [1]:
!pip install -q tensorflow-model-optimization pyyaml loguru model-profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00


In [2]:
!git clone -b dev https://github.com/SashaMogilevskii/model_compression

Cloning into 'model_compression'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 134 (delta 2), reused 13 (delta 1), pack-reused 118
Receiving objects: 100% (134/134), 78.37 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (51/51), done.
Updating files: 100% (27/27), done.


In [3]:
%cd model_compression/data

/content/model_compression/data


In [ ]:
!gdown --id 18gCWw8IkqbFr9X0TCBM5LCpsU11Ga6tc
!unrar x -y data.rar

In [5]:
%cd ../hw

/content/model_compression/hw


In [6]:
import sys
sys.path.append("../")

In [1]:
from box import Box
import time
import yaml

from model_profiler import model_profiler
import pandas as pd
import numpy as np
import tensorflow as tf
from torch.utils.data import DataLoader

from scr.utils.create_dataset import BirdDataset
from scr.utils.metrics import validation_epoch_end



In [2]:
with open("../scr/config_tf.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    config = Box(config)

In [3]:
df = pd.read_csv("../data/data.csv")
df_test = df[df.fold == 3].sample(n=100, random_state=42).reset_index(drop=True)
dataset_test = BirdDataset(
    df=df_test,
    path_to_folder_with_audio=config.path_to_files_base,
    tensorflow=True
)
valid_loader = DataLoader(
    dataset_test,
    batch_size=4,
    num_workers=config.num_workers
)


In [4]:
def get_metric_score(model):
    predicted_labels_list = None
    true_labels_list = None
    metric = validation_epoch_end
    start_time = time.time()

    for batch in valid_loader:
        X_batch = batch[0]
        y_b = batch[1]

        #convert to tf
        X_batch = tf.convert_to_tensor(X_batch.numpy(), dtype=tf.float32)
        y_batch = tf.convert_to_tensor(y_b.numpy(), dtype=tf.float32)

        logits = model.predict(X_batch)
        if predicted_labels_list is None:
                predicted_labels_list = logits
                true_labels_list = y_b
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, logits], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_b], axis=0)

    end_time = time.time()


    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0
    avg_metric = metric(all_true_labels, all_predicted_labels)
    t = end_time - start_time


    print("Наши метрики на нашей базовой модели:")
    print(f"Время работы модели на всем батче {t:.<2g} сек.")
    print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
    print("Метрики качества:")
    for m in avg_metric:
        print(f"metric {m} : {avg_metric[m]:.<5g}")

In [5]:
model = tf.keras.applications.efficientnet.EfficientNetB0(
    weights=None,
    include_top=False,
    input_shape=(626, 256, 3),
    drop_connect_rate=0.4,
)

input_shape = (626, 256, 3)

model = tf.keras.Sequential([
    tf.keras.Input(shape=input_shape),
    # *preprocessing_layers,
    model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(264, activation='sigmoid')])

existing_model = tf.keras.models.load_model('../models/model_tf')  # Замените на фактический путь к вашей модели
model.set_weights(existing_model.get_weights())


In [6]:
def get_model_size(model):
    param_size = 0
    for w in model.weights:
        param_size += np.prod(w.shape) * w.dtype.size
    size_all_mb = param_size / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))

def count_zero_weights(model):
    num_zero_weights = 0
    total_weights = 0

    for w in model.weights:
        num_zero_weights += np.sum(w.numpy() == 0)
        total_weights += np.prod(w.shape)

    sparsity = num_zero_weights / total_weights
    print(f"Разреженность весов: {sparsity * 100:.2f}%")
    return sparsity

In [7]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 4,
  'cluster_centroids_init': CentroidInitialization.DENSITY_BASED
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 20, 8, 1280)       7824007   
 )                                                               
                                                                 
 cluster_global_average_poo  (None, 1280)              0         
 ling2d (ClusterWeights)                                         
                                                                 
 cluster_flatten (ClusterWe  (None, 1280)              0         
 ights)                                                          
                                                                 
 cluster_dense (ClusterWeig  (None, 264)               676108    
 hts)                                                            
                                                                 
Total params: 8500115 (48.11 MB)
Trainable params: 43459

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 20, 8, 1280)       275395    
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 264)               264       
                                                                 
Total params: 275659 (1.05 MB)
Trainable params: 233636 (912.64 KB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


# Кластеризованная Модель

In [9]:
clustered_model_file = 'clustered_model_v1.tf'

clustered_model2 = tfmot.clustering.keras.strip_clustering(clustered_model)
clustered_model2.save(clustered_model_file)
loaded_model = tf.keras.models.load_model(clustered_model_file)

INFO:tensorflow:Assets written to: clustered_model_v1.tf/assets


INFO:tensorflow:Assets written to: clustered_model_v1.tf/assets


In [10]:
print("Характеристики кластеризованной модели")
print(model_profiler(loaded_model, 4))

Характеристики кластеризованной модели
| Model Profile                    | Value         | Unit    |
|----------------------------------|---------------|---------|
| Selected GPUs                    | None Detected | GPU IDs |
| No. of FLOPs                     | 0.0           | BFLOPs  |
| GPU Memory Requirement           | 0.0072        | GB      |
| Model Parameters                 | 4.3878        | Million |
| Memory Required by Model Weights | 16.738        | MB      |


In [11]:
print("Характеристики кластеризованной модели")
print(model_profiler(clustered_model, 4))

Характеристики кластеризованной модели
| Model Profile                    | Value         | Unit    |
|----------------------------------|---------------|---------|
| Selected GPUs                    | None Detected | GPU IDs |
| No. of FLOPs                     | 0.0           | BFLOPs  |
| GPU Memory Requirement           | 0.011         | GB      |
| Model Parameters                 | 8.5001        | Million |
| Memory Required by Model Weights | 32.4254       | MB      |


In [12]:
get_model_size(loaded_model)

model size: 16.738MB


In [13]:
get_model_size(clustered_model)

model size: 48.112MB


In [14]:
count_zero_weights(loaded_model)

Разреженность весов: 0.00%


2.279069820443484e-07

In [15]:
count_zero_weights(clustered_model)

Разреженность весов: 5.18%


0.051789652257645924

In [16]:
get_metric_score(clustered_model)

1/1 [==============================] - 0s 291ms/step
Наши метрики на нашей базовой модели:
Время работы модели на всем батче 18.4046 сек.
Время работы модели на одном сэмпле (AVG) 0.184 сек.
Метрики качества:
metric val_RMAP : 0.00378788


In [17]:
get_metric_score(loaded_model)

1/1 [==============================] - 0s 274ms/step
Наши метрики на нашей базовой модели:
Время работы модели на всем батче 12.8316 сек.
Время работы модели на одном сэмпле (AVG) 0.128 сек.
Метрики качества:
metric val_RMAP : 0.00378788


# Исходная модель

In [143]:
model_file = 'model_v1.tf'

model.save(model_file)

base_loaded_model = tf.keras.models.load_model(model_file)

In [144]:
print("Характеристики исходной модели")
print(model_profiler(base_loaded_model, 4))

Характеристики исходной модели
| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0     | BFLOPs  |
| GPU Memory Requirement           | 0.0072  | GB      |
| Model Parameters                 | 4.3878  | Million |
| Memory Required by Model Weights | 16.738  | MB      |


In [145]:
get_model_size(base_loaded_model)

model size: 16.738MB


In [146]:
count_zero_weights(base_loaded_model)

Разреженность весов: 0.00%


2.279069820443484e-07

In [147]:
get_metric_score(base_loaded_model)

1/1 [==============================] - 0s 42ms/step
Наши метрики на нашей базовой модели:
Время работы модели на всем батче 16.3301 сек.
Время работы модели на одном сэмпле (AVG) 0.163 сек.
Метрики качества:
metric val_RMAP : 0.0466699
